In [ ]:
import networkx as nx
import numpy as np
import random
import time
import matplotlib.pyplot as plt

def initialize_network(N, M, initial_defector_ratio):
    """
    Initialize a random graph with N nodes and M edges, assigning 'C' (cooperator) or 'D' (defector) states.
    Args:
        N (int): Number of nodes in the graph.
        M (int): Number of edges in the graph.
        initial_defector_ratio (float): Initial fraction of nodes to be defectors.
    Returns:
        G (networkx.Graph): Initialized graph with nodes assigned states.
    """
    # Create a random graph with N nodes and M edges
    G = nx.gnm_random_graph(N, M)
    # Calculate the number of defectors based on the initial ratio
    num_defectors = int(N * initial_defector_ratio)
    # Create a list of states: 'D' for defectors, 'C' for cooperators
    states = ['D'] * num_defectors + ['C'] * (N - num_defectors)
    # Shuffle states to assign them randomly
    random.shuffle(states)
    # Assign states to nodes
    for node, state in zip(G.nodes(), states):
        G.nodes[node]['state'] = state
    return G

def update_strategy(G, alpha, u, discordant_edges):
    """
    Update the state of a node along a discordant edge based on payoff difference.
    Args:
        G (networkx.Graph): The graph with nodes and their states.
        alpha (float): Sensitivity parameter for strategy update.
        u (float): Cost-to-benefit ratio.
        discordant_edges (list): List of edges where nodes have different states.
    Returns:
        discordant_edges (list): Updated list of discordant edges.
    """
    if discordant_edges:
        # Randomly select a discordant edge
        edge = random.choice(discordant_edges)
        # Identify cooperator (C) and defector (D) nodes in the edge
        c_node, d_node = edge if G.nodes[edge[0]]['state'] == 'C' else (edge[1], edge[0])
        # Calculate payoffs: C gets 1 per cooperating neighbor; D gets (1+u) per C, u per D
        payoff1 = sum(1 if G.nodes[n]['state'] == 'C' else 0 for n in G.neighbors(c_node))
        payoff2 = sum(1 + u if G.nodes[n]['state'] == 'C' else u for n in G.neighbors(d_node))
        # Compute transition probability: logistic if payoff difference <= 2, otherwise 0
        probability_1_to_2 = 1 / (1 + np.exp(alpha * (payoff1 - payoff2))) if (payoff1 - payoff2) <= 2 else 0
        # Update state based on probability
        if random.random() < probability_1_to_2:
            G.nodes[c_node]['state'] = 'D'
            changed_node = c_node
        else:
            G.nodes[d_node]['state'] = 'C'
            changed_node = d_node
        # Remove the processed edge from discordant_edges
        discordant_edges.remove(edge)
        # Update discordant edges for the changed node's neighbors
        for neighbor in G.neighbors(changed_node):
            potential_edge = tuple(sorted([changed_node, neighbor]))
            if G.nodes[changed_node]['state'] != G.nodes[neighbor]['state'] and potential_edge not in discordant_edges:
                discordant_edges.append(potential_edge)
            elif G.nodes[changed_node]['state'] == G.nodes[neighbor]['state'] and potential_edge in discordant_edges:
                discordant_edges.remove(potential_edge)
    return discordant_edges

def rewire(G, discordant_edges, strategy_type, p, threshold):
    """
    Rewire a discordant edge based on the specified strategy, updating the discordant edge list.
    Args:
        G (networkx.Graph): The graph with nodes and their states.
        discordant_edges (list): List of edges where nodes have different states.
        strategy_type (str): Rewiring strategy ('max_degree_C', 'min_degree_C', 'probabilistic', 'random', 'hybrid').
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy to switch between max_degree_C and min_degree_C.
    Returns:
        discordant_edges (list): Updated list of discordant edges.
    """
    if discordant_edges:
        # Select a discordant edge
        edge = random.choice(discordant_edges)
        # Identify cooperator (C) and defector (D) nodes in the edge
        c_node, d_node = edge if G.nodes[edge[0]]['state'] == 'C' else (edge[1], edge[0])
        # Remove the edge from the graph and discordant_edges
        G.remove_edge(*edge)
        discordant_edges.remove(edge)
        # Find nodes not currently connected to c_node (excluding itself)
        potential_nodes = [node for node in G.nodes() if not G.has_edge(c_node, node) and node != c_node]
        new_node = None

        # For hybrid strategy, choose sub-strategy based on cooperation fraction
        if strategy_type == 'hybrid':
            coop_fraction = calculate_cooperation_fraction(G)
            if coop_fraction < threshold:
                strategy_type = 'max_degree_C'
            else:
                strategy_type = 'min_degree_C'

        # Rewiring strategies
        if strategy_type == 'max_degree_C':
            max_degree = -1
            for node in potential_nodes:
                if G.nodes[node]['state'] == 'C' and G.degree[node] > max_degree:
                    max_degree = G.degree[node]
                    new_node = node
        elif strategy_type == 'min_degree_C':
            min_degree = float('inf')
            for node in potential_nodes:
                if G.nodes[node]['state'] == 'C' and G.degree[node] < min_degree:
                    min_degree = G.degree[node]
                    new_node = node
        elif strategy_type == 'probabilistic':
            if random.random() < p:
                cooperators = [node for node in potential_nodes if G.nodes[node]['state'] == 'C']
                new_node = random.choice(cooperators) if cooperators else random.choice(potential_nodes)
            else:
                new_node = random.choice(potential_nodes)
        elif strategy_type == 'random':
            if potential_nodes:
                new_node = random.choice(potential_nodes)

        # Fallback: if no specific node selected, choose randomly
        if new_node is None and potential_nodes:
            new_node = random.choice(potential_nodes)
        if new_node:
            # Add new edge and update discordant edges
            G.add_edge(c_node, new_node)
            for neighbor in G.neighbors(c_node):
                potential_edge = tuple(sorted([c_node, neighbor]))
                if G.nodes[c_node]['state'] != G.nodes[neighbor]['state'] and potential_edge not in discordant_edges:
                    discordant_edges.append(potential_edge)
                elif G.nodes[c_node]['state'] == G.nodes[neighbor]['state'] and potential_edge in discordant_edges:
                    discordant_edges.remove(potential_edge)
    return discordant_edges

def simulate(G, alpha, u, w, max_times, strategy_type, p, threshold):
    """
    Simulate network evolution until max iterations or no discordant edges remain.
    Args:
        G (networkx.Graph): The graph to simulate.
        alpha (float): Sensitivity parameter for strategy update.
        u (float): Cost-to-benefit ratio.
        w (float): Probability of strategy update versus rewiring.
        max_times (int): Maximum number of iterations.
        strategy_type (str): Rewiring strategy.
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy.
    """
    iteration = 0
    # Initialize list of discordant edges (edges between C and D nodes)
    discordant_edges = [(i, j) for i, j in G.edges() if G.nodes[i]['state'] != G.nodes[j]['state']]
    # Continue until max iterations or no discordant edges remain
    while iteration < max_times and any(G.nodes[i]['state'] != G.nodes[j]['state'] for i, j in G.edges()):
        if random.random() < w:
            update_strategy(G, alpha, u, discordant_edges)  # Update strategy
        else:
            rewire(G, discordant_edges, strategy_type, p, threshold)  # Rewire edge
        iteration += 1

def calculate_cooperation_fraction(G):
    """
    Calculate the fraction of cooperating nodes in the network.
    Args:
        G (networkx.Graph): The graph with nodes and their states.
    Returns:
        float: Fraction of nodes that are cooperators.
    """
    cooperators = sum(1 for node in G.nodes() if G.nodes[node]['state'] == 'C')
    return cooperators / len(G.nodes())

def simulate_and_average_cooperation(N, M, initial_defector_ratio, alpha, u, w, simulations, max_times, strategy_type, p, threshold):
    """
    Run multiple simulations and return the average fraction of cooperators.
    Args:
        N (int): Number of nodes.
        M (int): Number of edges.
        initial_defector_ratio (float): Initial fraction of defectors.
        alpha (float): Sensitivity parameter for strategy update.
        u (float): Cost-to-benefit ratio.
        w (float): Probability of strategy update versus rewiring.
        simulations (int): Number of simulations to run.
        max_times (int): Maximum iterations per simulation.
        strategy_type (str): Rewiring strategy.
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy.
    Returns:
        float: Average fraction of cooperators across simulations.
    """
    total_cooperation = 0
    for _ in range(simulations):
        G = initialize_network(N, M, initial_defector_ratio)
        simulate(G, alpha, u, w, max_times, strategy_type, p, threshold)
        total_cooperation += calculate_cooperation_fraction(G)
    return total_cooperation / simulations

def run_simulation_for_different_w(w_values, N, M, initial_defector_ratio, alpha, u_values, simulations, max_times, strategy_types, p, threshold):
    """
    Run simulations for different w values, u values, and strategy types, returning results in a dictionary.
    Args:
        w_values (list): List of strategy updating probabilities to test.
        N (int): Number of nodes.
        M (int): Number of edges.
        initial_defector_ratio (float): Initial fraction of defectors.
        alpha (float): Sensitivity parameter for strategy update.
        u_values (list): List of cost-to-benefit ratios to test.
        simulations (int): Number of simulations per condition.
        max_times (int): Maximum iterations per simulation.
        strategy_types (list): List of rewiring strategies to test.
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy.
    Returns:
        dict: Dictionary mapping (strategy_type, w) to a list of cooperation fractions for each u.
    """
    results = {}
    start_time = time.time()
    for strategy_type in strategy_types:
        for w in w_values:
            cooperation_fractions = []
            for u in u_values:
                avg_cooperation = simulate_and_average_cooperation(
                    N, M, initial_defector_ratio, alpha, u, w, simulations, max_times, strategy_type, p, threshold
                )
                cooperation_fractions.append(avg_cooperation)
            results[(strategy_type, w)] = cooperation_fractions
    print(f"Total time for all simulations: {time.time() - start_time:.2f} seconds")
    return results

def plot_multiple_cooperation_vs_u(u_values, results):
    """
    Plot the fraction of cooperators versus cost-to-benefit ratio u for different strategies and w values.
    Args:
        u_values (list): List of cost-to-benefit ratios.
        results (dict): Dictionary mapping (strategy_type, w) to a list of cooperation fractions.
    """
    for (strategy_type, w), cooperation_fractions in results.items():
        plt.plot(u_values, cooperation_fractions, marker='o', label=f'{strategy_type}, w={w}')
    plt.xlabel('Cost-to-benefit ratio, u')  # Label for x-axis
    plt.ylabel('Fraction of cooperators')  # Label for y-axis
    plt.ylim(0, 1)  # Set y-axis limits
    plt.legend()  # Display legend
    plt.show()  # Display the plot

# -------------------------------
# Simulation example parameters
if __name__ == "__main__":
    N = 10000  # Number of nodes
    M = 50000   # Number of edges
    initial_defector_ratio = 0.5  # Initial fraction of defectors
    alpha = 30                  # Sensitivity parameter for strategy update
    u_values = np.linspace(0, 1, 51)  # Range of cost-to-benefit ratios
    w_values = [0.1]           # Probability weight for strategy update vs. rewiring
    simulations = 1             # Number of simulation runs per condition
    max_times = 10**12       # Maximum iterations per simulation
    strategy_types = ['min_degree_C']  # Rewiring strategy to test
    p = 1                       # Probability for probabilistic rewiring
    threshold = 0.5             # Threshold for hybrid strategy

    # Run simulations for different w values, u values, and strategies
    results = run_simulation_for_different_w( w_values, N, M, initial_defector_ratio, alpha, u_values, simulations, max_times, strategy_types, p, threshold )
    print(results)
    plot_multiple_cooperation_vs_u(u_values, results)